In [1]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 3.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 kB 3.8 MB/s eta 0:00:0000:0100:01
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 2.

In [5]:
#GET INTRADAY DATA
import pandas as pd
import numpy as np
import datetime
import seaborn as sn
import matplotlib.pyplot as plt
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def GetData_Intraday(symbol):
  url = f'https://s.cafef.vn/Lich-su-giao-dich-{symbol}-6.chn#data'
  data = pd.read_html(url)
  return data

def GetSticks_Intraday(symbol):
  url = f'https://s.cafef.vn/Lich-su-giao-dich-{symbol}-6.chn#data'
  data = pd.read_html(url)
  
  df_sticks = data[len(data)-1]
  df_sticks = pd.DataFrame(df_sticks.values,columns=['Time','PriceX','Volume','Sum Volume','Percent'])

  #Tách giá
  df_sticks['Price'] = df_sticks['PriceX'].apply(lambda x: x.split(' ')[0])
  df_sticks['Price']

  df_sticks['%'] = df_sticks['PriceX'].apply(lambda x: x.split(' ')[2])
  df_sticks['%'] = df_sticks['%'].apply(lambda x: x[1:len(x)-2])

  del df_sticks['PriceX']

  #cols = ['Volume','Sum Volume','Price','%','Percent']
  cols = ['Volume','Sum Volume','Price','%']
  df_sticks[cols] = df_sticks[cols].apply(pd.to_numeric, downcast='float', errors='coerce')



  df_sticks['Time'] = df_sticks['Time'].apply(lambda x: datetime.datetime.today().strftime('%Y-%m-%d') + ' ' + str(x))
  df_sticks['Time'] = df_sticks['Time'].astype("datetime64")
  df_sticks.set_index('Time')

  df_sticks['Money'] = df_sticks['Price']*df_sticks['Volume']

  return df_sticks

def FindShark(symbol):
  df = GetSticks_Intraday(symbol=symbol)
  sum_money = np.sum(df['Money']) #.apply(lambda x: x.split(' ')[0])
  df['Shark'] = df['Money'].apply(lambda x: True if x >= (sum_money*1)/100 else False)

  df_view = df
  df_view = df_view[df_view['Shark'] == True]
  df_view['Money'] = df_view['Money'].apply("{0:,.2f}".format)  
  return df_view

In [6]:
#GET INTRADAY DATA
import pandas as pd
import numpy as np
import datetime
import seaborn as sn
import matplotlib.pyplot as plt
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

class Intraday:
      
      def __init__(self, symbol) -> None:
            self.Symbol = symbol
            self.Url = f'https://s.cafef.vn/Lich-su-giao-dich-{self.Symbol}-6.chn#data'
            
            self.data = pd.DataFrame()
            self.df_sticks = pd.DataFrame()
      
      def LoadData(self):
            data = pd.read_html(self.Url)
            self.data = data
            self.df_sticks = self.GetSticks()
            
      def GetSticks(self):
            if (self.data == None):
                  self.LoadData()
                              
            data = self.data
            
            df_sticks = data[len(data)-1]
            df_sticks = pd.DataFrame(df_sticks.values,columns=['Time','PriceX','Volume','Sum Volume','Percent'])

            #Tách giá
            df_sticks['Price'] = df_sticks['PriceX'].apply(lambda x: x.split(' ')[0])
            df_sticks['Price']

            df_sticks['%'] = df_sticks['PriceX'].apply(lambda x: x.split(' ')[2])
            df_sticks['%'] = df_sticks['%'].apply(lambda x: x[1:len(x)-2])

            del df_sticks['PriceX']

            #cols = ['Volume','Sum Volume','Price','%','Percent']
            cols = ['Volume','Sum Volume','Price','%']
            df_sticks[cols] = df_sticks[cols].apply(pd.to_numeric, downcast='float', errors='coerce')



            df_sticks['Time'] = df_sticks['Time'].apply(lambda x: datetime.datetime.today().strftime('%Y-%m-%d') + ' ' + str(x))
            df_sticks['Time'] = df_sticks['Time'].astype("datetime64")
            df_sticks.set_index('Time')

            df_sticks['Money'] = df_sticks['Price']*df_sticks['Volume']

            return df_sticks

      def FindShark(self):
            df = GetSticks_Intraday(symbol=self.Symbol)
            sum_money = np.sum(df['Money']) #.apply(lambda x: x.split(' ')[0])
            df['Shark'] = df['Money'].apply(lambda x: True if x >= (sum_money*1)/100 else False)

            df_view = df
            df_view = df_view[df_view['Shark'] == True]
            df_view['Money'] = df_view['Money'].apply("{0:,.2f}".format)  
            return df_view
      
      def ExplainTalk(self):
            max = np.max(self.df_sticks['Price'])
            min = np.min(self.df_sticks['Price'])
            
            output = f'Giá cao nhất: {max} - Giá thấp nhất {min}'
            return output
      
      def Draw_Chart_Sticks(self):            
            plt.title(self.Symbol)
            plt.plot(self.df_sticks['Time'],self.df_sticks["Price"])
            plt.legend()
            plt.show()

In [7]:
d = Intraday(symbol='HDG')
d.LoadData()
df = d.FindShark()
print(df)
#d.Draw_Chart_Sticks()
#d.ExplainTalk()

                   Time  Volume  Sum Volume   Percent      Price     %  \
4   2022-08-01 13:50:22  2000.0    178420.0  1.12 (%)  52.599998 -0.57   
108 2022-08-01 13:26:20  6120.0    161620.0  3.43 (%)  52.799999 -0.19   
109 2022-08-01 13:25:44  2550.0    155500.0  1.43 (%)  52.900002  0.00   
142 2022-08-01 13:18:01  2000.0    145100.0  1.12 (%)  53.000000  0.19   
373 2022-08-01 10:36:52  2050.0    106460.0  1.15 (%)  52.599998 -0.57   
447 2022-08-01 10:22:42  1990.0     91000.0  1.11 (%)  52.900002  0.00   
593 2022-08-01 09:59:27  3700.0     57010.0  2.07 (%)  54.000000  2.08   
665 2022-08-01 09:44:36  2000.0     40550.0  1.12 (%)  54.000000  2.08   
677 2022-08-01 09:42:55  2000.0     37290.0  1.12 (%)  54.500000  3.02   
699 2022-08-01 09:41:03  2000.0     32650.0  1.12 (%)  54.000000  2.08   
707 2022-08-01 09:40:21  4000.0     30270.0  2.24 (%)  54.000000  2.08   
710 2022-08-01 09:39:52  1930.0     26200.0  1.08 (%)  53.900002  1.89   
726 2022-08-01 09:38:00  4760.0     22

/var/folders/jg/r60w3sys1539n6tc4gg33dnw0000gn/T/ipykernel_81277/381342782.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_view['Money'] = df_view['Money'].apply("{0:,.2f}".format)


In [ ]:
#from datetime import time
from time import sleep


stocks = ['VND','TPB','FRT']
for symbol in stocks:
    d = Intraday(symbol=symbol)
    d.LoadData()
    d.Draw_Chart_Sticks()
    d.ExplainTalk()
    sleep(3)

**FIRST LOOK**

Có 02 bảng dữ liệu:
1. Bảng liệt kê các mức giá của dữ liệu với khối lượng lệnh và tỷ trọng phần trăm
2. Bảng chi tiết từng stick lệnh

Như vậy ta tách thành 02 bộ dữ liệu:
data_sticks : Là danh sách từng lệnh chi tiết
data_prices : Là dữ liệu về từng mức giá

PHÂN TÍCH STICKS LỆNH

1. Thời gian đã được sắp xếp theo thứ tự chưa ?
2. Liệt kê các lệnh có khối lượng lớn


KHAI PHÁ DỮ LIỆU



1.   Có bao nhiêu lệnh lớn xuất hiện ?
- Lệnh lớn: Là lệnh mà có giá trị giao dịch chiếm khoảng 1% tổng giá trị giao dịch trong ngày (hoặc ta có thể định nghĩa khác
2.   List item

